In [1]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(10)

In [2]:
t.ChangeVocab(v2)
v2.InitUnigramTable()

vocab_size = v2.size
dim = 32

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn2 = nn.Linear(dim,dim)
syn2.weights.uniform(seed)
syn2.weights -= 0.5
syn2.weights /= dim

syn1neg = nn.SparseLinearOutput(dim,vocab_size)
syn1neg.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.Sigmoid(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

In [3]:
sampler = nn.Sampler(v2,5)

In [4]:
words = 0
for sent in t.sentences:
    for word in sent:
        words+=1

In [5]:
cbow = nn.CBOW(t.sentences,v2,sampler,5,5)

In [6]:
cbow.getTargetValuesReference().get()

(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [7]:
cbow.next()

In [8]:
list(cbow.getOutputIndicesReference())

[162, 2091, 21, 3738, 5, 324]

In [9]:
optim.train(cbow,0.01,1,1)

-1.0

In [10]:
%%timeit
optim.train(cbow,0.01,1,1)

1 loop, best of 3: 2.67 s per loop


In [10]:
%%timeit
optim.train(cbow,0.01,1,4)

1 loop, best of 3: 1.44 s per loop


In [ ]:
%%timeit
optim.train(cbow,0.01,3,4)

In [ ]:
words

In [11]:
import numpy as np

word2vec = {}
vocab = list()
for i in range(v2.size):
    l2 = np.sqrt(syn0.weights[i].dot(syn0.weights[i]))
    v = syn0.weights[i]
    v /= l2
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]
    
sim = Counter()
for word in vocab:
    sim[word] = word2vec['ron'].dot(word2vec[word])
sim.most_common()

[('ron', 0.9999997615814209),
 ('hermione', 0.8837041854858398),
 ('ginny', 0.8498551249504089),
 ('hagrid', 0.8308994174003601),
 ('neville', 0.7926112413406372),
 ('lupin', 0.7678225040435791),
 ('luna', 0.7520603537559509),
 ('she', 0.730151891708374),
 ('snape', 0.7165758013725281),
 ('lavender', 0.7090650796890259),
 ('tonks', 0.6988118290901184),
 ('parvati', 0.6966623067855835),
 ('cho', 0.6906642317771912),
 ('fleur', 0.6898412108421326),
 ('moody', 0.6858638525009155),
 ('bill', 0.6779994368553162),
 ('bagman', 0.6751066446304321),
 ('seamus', 0.6671534180641174),
 ('mundungus', 0.6607690453529358),
 ('quickly', 0.6606795787811279),
 ('malfoy', 0.660637378692627),
 ('krum', 0.6588022112846375),
 ('fred', 0.657082200050354),
 ('colin', 0.655830442905426),
 ('umbridge', 0.6557586789131165),
 ('cedric', 0.6480444669723511),
 ('dumbledore', 0.6426758170127869),
 ('dean', 0.6411870718002319),
 ('karkaroff', 0.6407816410064697),
 ('ernie', 0.6363554000854492),
 ('fudge', 0.635504543

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)